In [129]:
# coding: utf-8
# citations: https://pythonmana.com/2021/03/20210329161147051K.html
# citation: https://stackoverflow.com/questions/43637211/retrieve-document-content-with-document-structure-with-python-docx

# Import Libraries
!pip install docx
from docx import Document
from docx.document import Document as _Document
from docx.oxml.text.paragraph import CT_P
from docx.oxml.table import CT_Tbl
from docx.table import _Cell, Table
from docx.text.paragraph import Paragraph
import pandas as pd
import numpy as np
import re
from os.path import exists
import logging 
!pip install xlrd

In [130]:
# Configure Logger
logging.basicConfig(filename='std.log', 
                    filemode='w', 
                    format='%(name)s - %(levelname)s - %(asctime)s:  %(message)s',
                    level = logging.DEBUG, force=True, datefmt='%Y-%m-%d %H:%M:%S')


logger=logging.getLogger()

In [131]:
# Master File
master = 'Master.xlsx' 

# Project Settings
try:
    logger.info('Trying to read the "project settings tab" from the Master file')
    project_settings = pd.read_excel(master, sheet_name='project settings', header=None)
    logging.info('Printout of project settings data: {}'.format(project_settings.head()))

except IOError as e:
    logger.exception('Is your Master File open? It should be closed to run this code.' + str(e), exc_info=False)
except ValueError as e:
    logger.exception("Can't find the Project Settings worksheet. Check the name of your tabs in Master.xlsx." + str(e), exc_info=False)
                     
# Table Settings
try:
    logger.info('Trying to read the "table settings tab" from the Master file')
    table_settings = pd.read_excel(master, sheet_name='table settings')
    logging.info('Printout of table settings data: {}'.format(table_settings.head()))
except IOError as e:
    logger.exception('Is your Master File open? It should be closed to run this code.' + str(e), exc_info=False)
except ValueError as e:
    logger.exception("Can't find the Table Settings worksheet. Check the name of your tabs in Master.xlsx." + str(e), exc_info=False)

# Word Table Shells INPUT Filename
try:
    logger.info('Trying to read the Word "Table Shells INPUT Filename" from the Master file')
    document = Document(project_settings.iloc[0,1].strip())
    print(document)
    logging.info('Word Table Shells INPUT Filename: {}'.format(report_doc))
except Exception as e:
    logger.exception('You need to fill in the "Word Table Shells INPUT Filename" on the Project Settings tab."' + str(e), exc_info=False)

if (not report_doc):
    logger.exception('You need to fill in the "Word Table Shells INPUT Filename" on the Project Settings tab.')

In [134]:
########################################################################
# Load Project and Table Settings 
########################################################################

# Master File
master = 'Master.xlsx' 

# Project Settings
project_settings = pd.read_excel(master, sheet_name='project settings', header=None)

# Table Settings
table_settings = pd.read_excel(master, sheet_name='table settings')

# Word Table Shells INPUT Filename
document = Document(project_settings.iloc[0,1].strip())

# Word Table Shells OUTPUT Filename
output_doc = project_settings.iloc[1,1].strip()
print(output_doc)

# Table Format Style
table_style = project_settings.iloc[2,1].strip()
print(table_style)

# Table Caption Style
caption_style = project_settings.iloc[3,1].strip()
print(caption_style)

results.docx
__Table Style-AIR 2021
Exhibit Title


In [133]:
########################################################################
# Populate Table Data in Word
########################################################################

# Loop through the paragraphs & table pairs in the Word document
#citation: source: https://theprogrammingexpert.com/write-table-fast-python-docx/
''' Define script to identify table 'child' within paragraph 'parent' based on document order
    Each returned value is an instance of either Table or Paragraph. '''
def iter_block_items(parent):
    if isinstance(parent, _Document):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        raise ValueError("something's not right")

    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            yield Paragraph(child, parent)
        elif isinstance(child, CT_Tbl):
            yield Table(child, parent)      

'''Identifies table meta-data from crosswalk file'''
def findtables(document, table_settings):

    # Iterate through paragraphs
    for block in iter_block_items(document):

        # Identify paragraphs
        if isinstance(block, Paragraph):

            # Identify table captions by finding text with caption_style formatting
            if block.style.name==caption_style:

                print(f"blocktext: {block.text}")

                # Find the Exhibit Label in the Word INPUT file
                exhibit_name = block.text 
                label = exhibit_name[(re.search(r'[.:]', exhibit_name)).start() + 2:]
                print(label)
                      
                # Load corresponding table data
                # Should it be be .loc rather than .iloc so not sensitive to column ordering?
                worksheet_name = table_settings.loc[
                    table_settings[
                        'word_table_title'].str.contains(
                        label, case=False)].iloc[0, 1]           

                workbook_name = table_settings.loc[
                    table_settings[
                        'word_table_title'].str.contains(
                        label, case=False)].iloc[0, 2]
                
                tables_data = pd.read_excel(
                    workbook_name, sheet_name=worksheet_name, header=None)
                
                print(f"workbook_name: {workbook_name}")

                # Load "nofill_first_x_rows" setting
                skip_table_rows = table_settings.loc[
                    table_settings[
                        'word_table_title'].str.contains(
                        label, case=False)].iloc[0, 3]
                
                # Load "nofill_first_y_cols" setting
                skip_table_cols = table_settings.loc[
                    table_settings[
                        'word_table_title'].str.contains(
                        label, case=False)].iloc[0, 4]
                
                print(skip_table_rows)
                print(skip_table_cols)
                
                # Load "skip_merged_rows" setting
                skip_merged_rows = table_settings.loc[
                    table_settings[
                        'word_table_title'].str.contains(
                        label, case=False)].iloc[0, 5]
                print(skip_merged_rows)
                
                # Move on to the corresponding table object
                continue

        # Load corresponding table object
        elif isinstance(block,Table):
            
            tablepopulate(block, tables_data, skip_table_rows, skip_table_cols, skip_merged_rows)

'''Extracts that table's formatting specifications and fills in the data'''
def tablepopulate(block, df, skip_table_rows, skip_table_cols, skip_merged_rows):
        print(df.head())
        
        # loop through rows and cols of the dataframe to populate the table object
        for i in range(skip_table_rows, df.shape[0]):
            for j in range(skip_table_cols, df.shape[1]):
                
                # Skip merged cells in the first column
                if j == 0 and skip_merged_rows == 'y':
                    c=block.cell(i,0)

                    if c._tc.right > 1:
                        print(j)
                        continue
                                      
                # Skip over blank (nan) cells in the dataframe
                if (str(df.values[i,j])) != 'nan':
                    #print(str(df.values[i,j]))
                    block.cell(i,j).text = str(df.values[i,j])
        
        # Add table styles and formats                        
        block.style = table_style

findtables(document, table_settings)

#3. Save the outputted Word document
#document.save(output_doc)

document.save('result.docx')

blocktext: Exhibit . I am first
I am first
workbook_name: rawdata1_mess.xlsx
0
0
y
          0      1      2      3
0  Response    Yes     No  Maybe
1     Total  0.028  0.012  0.026
2   Group A    NaN    NaN    NaN
3    Site 1  0.095  0.106  0.097
4    Site 2  0.004      0  0.004
0
0
blocktext: Exhibit . I am second
I am second
workbook_name: rawdata1_mess.xlsx
0
0
y
                    0                 1       2       3       4       5
0              Status           Outcome  Site 1  Site 2  Site 3  Site 4
1  Education Outcomes  HS Degree Earned    0.41    0.37    0.41    0.37
2                 NaN        GED Earned    0.59    0.63    0.59    0.63
3    Program Outcomes            Exiter    0.41    0.37    0.41    0.37
4                 NaN         Completer    0.59    0.63    0.59    0.63
blocktext: Exhibit . I am third
I am third
workbook_name: rawdata1_mess.xlsx
0
0
y
           0            1     2                3     4
0     Status  Participant   NaN  Non-Participant   NaN
1    

In [ ]:
                #print(f": i {i} j {j}")
                #print(pd.isna(df[i][j]))
                
#                 #Check for merged cells in the first column
#                 if j == 0:
#                     c=table.cell(i,0)
#                     print(c._tc.right)
#                     print(c.text)
#                     if c._tc.right > 1:
#                         continue
#                     block.cell(i,j).text = str(df.values[i,j])
                
#                 else:
#                     block.cell(i,j).text = str(df.values[i,j])
                    
#                     # c=table.cell(1,0)
# # print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)
                                       

In [ ]:

#             except:

#                 print("f({exhibit_name} does not exist in the Excel file."
#                       f"Please double-check your Excel file information for typos")


#             print(f"first column {block.row.cells[0].text}")
#             print(f"first row {block.columns.cells[0].text}")

#             row = block.rows[0]
#             # Access the first column of the row
#             row.cells[0].text
#             # Access the second column of the row
#             row.cells[1].text = 'This is the second row, second column'

#             block.rows[0].style = col_heading_style  # first column heading style
#             block.columns[0].style = row_heading_style   # first row heading style
              
              #                 value = df.at[i, j] #get cell value
#                 print(np.isnan(value), end="\t")
#                 if pd.isna(df[i][j]):                    
#                     print(df.values[i,j])

In [ ]:
# table=document.tables[0]
# row = table.rows[0]
# len(row.cells)
# len(Column.cells

# c=table.cell(0,0)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(1,0)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(2,0)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(3,0)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(4,0)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(5,0)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)




# c=table.cell(0,1)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(1,1)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(2,1)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(3,1)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(4,1)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(5,1)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

In [ ]:
        # Set the initial row for iteration in the dataframe (default = 0)
        initial_row = 0
        if overwrite_table_row == 'n': 
            initial_row = 1 
        print(f"initial_row {initial_row}")

        # Set the initial col for iteration in the dataframe (default = 0)
        initial_col = 0
        if overwrite_table_col == 'n': 
            initial_col = 1
        print(f"initial_col {initial_col}")

In [ ]:
#                 # Overwrite Settings
#                 overwrite_table_row = crosswalk_data.loc[crosswalk_data['word_table_title'] == exhibit_name].iloc[0, 3]
#                 overwrite_table_col = crosswalk_data.loc[crosswalk_data['word_table_title'] == exhibit_name].iloc[0, 4]


# # 2. Load in all sheets from Excel Workbook(s) into Pandas Dataframe
# tables_data = pd.read_excel(excel_doc, sheet_name=None, header=None)
# print(tables_data["Table 1"])


# Row Heading Styles in Word. Replace the default if needed.
row_heading_style = 'Table 11 Row Heading'

# Column Heading Styles in Word. Replace the default if needed.
col_heading_style = 'Table 11 Column Heading'

exhibit_name = "Exhibit 1. I am first"
excel_sheet_name = crosswalk_data.loc[crosswalk_data['word_table_title'] == exhibit_name].iloc[0, 1]
print(f"excel_sheet_name: {excel_sheet_name}")


# Are all your data tables in a single Excel workbook (1 table per worksheet?) Enter 'y' for yes and 'n' for no.
# If you are pulling data from multiple workbooks, populate that information in the Crosswalk file
single_workbook = 'y'

# Excel Document Name (If your Excel doc is rawdata.xlsx, excel_doc = 'rawdata.xlsx')
excel_doc = 'rawdata1_mess.xlsx'



path = project_settings.iloc[0,1]
#path2 = path.replace("\", "\\")
print(path)
                     #"C:\Users\miche\Documents\school\jaar2\MIK\2.6\vektis_agb_zorgverlener"
# path = rC:\Users\jstockham\OneDrive - American Institutes for Research in the Behavioral Sciences\JSmith\PYTHON_CODE_2022\table_automate_jess\general_table_automate'
# path = r'C:\Users\jstockham\OneDrive - American Institutes for Research in the Behavioral Sciences\JSmith\PYTHON_CODE_2022\table_automate_jess\general_table_automate'

#path = 'C:/Users/jstockham/OneDrive - American Institutes for Research in the Behavioral Sciences/JSmith/PYTHON_CODE_2022/table_automate_jess/general_table_automate/'

print(path)
# 1. Load Crosswalk data into Pandas Dataframe
table_settings = pd.read_excel(master, sheet_name="table settings")
#print(crosswalk_data.head())

pathlink = r"C:\Users\jstockham\OneDrive - American Institutes for Research in the Behavioral Sciences\JSmith\PYTHON_CODE_2022\table_automate_jess\general_table_automate"
print(pathlink)





# pathlink + '\' + '
# # 3. Load Word Document into a docx file object
# document = Document(report_doc)

# #4. Set output to be the same as the input Word file
# #print(report_doc)
# if not output_doc:
#     output_doc = report_doc

                    print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)